In [ ]:
from scorers.answer_format import Answer_format_Scorer
from scorers.Input_dependent_query import Input_dependent_Scorer
from scorers.Task_prescribed_phrases import Task_phrases_Scorer
from scorers.count_limit import Count_Limit_Scorer
import argparse
import utils
import glob
import numpy as np
import pandas as pd
import os
import re
import copy
import pdb
import warnings


Format_evaluator = Answer_format_Scorer()
Input_evaluator = Input_dependent_Scorer()
Task_evaluator = Task_phrases_Scorer()
Count_evaluator = Count_Limit_Scorer()


warnings.filterwarnings("ignore")

def get_score_each_task(res_list, labeled_data, interset_sample_ids):
    score_list = []
    matched_ins = []
    for sample in res_list:
        # interset_sample_ids = ['5df63325dafc3b787adb65c7108ac41ed71244c847019d35f15021620a83ffe8']
        if sample['Instruction_id'] not in interset_sample_ids:
            continue
        
        try:
            matched_sample = list(filter(lambda x: x['Instruction_id'] == sample['Instruction_id'], labeled_data))[0]
        except:
            continue

        

        format_score = Format_evaluator.get_final_score(sample['model_answer'], sample['histories'], matched_sample['Answer_format'])
        input_score = Input_evaluator.get_final_score(sample['model_answer'], sample['histories'], sample['input_text'], matched_sample['Input_depdent_query'])
        task_score = Task_evaluator.get_final_score(sample['model_answer'], sample['histories'], matched_sample['Task_prescribed_phrases'])
        count_limit_score = Count_evaluator.get_final_score(sample['model_answer'], sample['histories'], matched_sample['Count_limit'], matched_sample["lang"])
        # print(sample['model_answer'])

        score_list.append(input_score)
        matched_ins.append(sample['Instruction_id'])

    return score_list, matched_ins

In [19]:
# task_name = ''
# task_name = 'closed_qa'
task_name = 'meta_prompt'
model1 = 'chatgpt'
model2 = 'gpt4'
# model1 = 'chatgpt'
# model2 = 'gpt4'

res_list_1 = utils.readjson('../results/' + model1 + '/res_' + task_name + '.json')
res_list_2 = utils.readjson('../results/' + model2 + '/res_' + task_name + '.json')
labeled_data = utils.readjson('../data/' + task_name + '.json')

In [20]:
set_list = [set([i['Instruction_id'] for i in res_list_1]),set([i['Instruction_id'] for i in res_list_2])]
interset_sample_ids = set.intersection(*set_list)

In [21]:
len(interset_sample_ids)

27

In [22]:
# score_list_1 = get_score_each_task(res_list_1[9:12], labeled_data[9:12])
# score_list_2 = get_score_each_task(res_list_2[9:12], labeled_data[9:12])
score_list_1, matched_ins_1 = get_score_each_task(res_list_1, labeled_data, interset_sample_ids)
score_list_2, matched_ins_2 = get_score_each_task(res_list_2, labeled_data, interset_sample_ids)

In [23]:
matched_ins_1[0], matched_ins_2[0]

('e8a6a40d816225a350fe0dc1e66f194333cf65784d9366af90767b770557e06f',
 'e8a6a40d816225a350fe0dc1e66f194333cf65784d9366af90767b770557e06f')

In [24]:
len(matched_ins_1), len(matched_ins_2)

(20, 20)

In [30]:
for i in range(len(matched_ins_1)):
    score1 = score_list_1[i]
    score2 = score_list_2[matched_ins_2.index(matched_ins_1[i])]
    # if score1 < score2 and score2 < 1:
    if score2 > 0.8:
    # if np.nanmean(score_list_2[i]) < 0.8:
        matched_sample = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], labeled_data))[0]
        res1 = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], res_list_1))[0]
        res2 = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], res_list_2))[0]
        print('====')
        print(i)
        print(matched_sample['Instruction'])
        print(matched_sample['Instruction_id'])
        print('#############')
        print(res1['model_answer'])
        print('***')
        for ll in matched_sample['Input_depdent_query'][0]['limit']:
            if ll not in res1['model_answer']:
                print(ll)
        print('#############')
        print(res2['model_answer'])
        print('***')
        for ll in matched_sample['Input_depdent_query'][0]['limit']:
            if ll not in res2['model_answer']:
                print(ll)
        print('#############')
        print(matched_sample['Answer_format'])
        print(matched_sample['Input_depdent_query'][0]['limit'])
        print(matched_sample['Task_prescribed_phrases'])
        print(matched_sample['Count_limit'])

        print()
        print(score_list_1[i])
        print(score_list_2[matched_ins_2.index(matched_ins_1[i])])
        # print(score_list_2[i])
        print()

====
0
模仿以下格式，出一道题目：
示例如下
包含单位：“千克”
"input": "一个物体质量为2千克，以10米/秒的速度运动，它的动能是多少焦耳？\"
"output": "动能公式为：E=1/2mv^2，其中E表示动能，m表示物体质量，v表示物体速度。
将题目中的数据代入公式，得到：
E=1/2 x 2千克 x (10米/秒)^2 = 100焦耳\n因此，这个物体的动能是100焦耳。"
"answer": \"100\"
包含单位：“盎司”
e8a6a40d816225a350fe0dc1e66f194333cf65784d9366af90767b770557e06f
#############
"input": "一个物体质量为4盎司，以8英尺/秒的速度运动，它的动能是多少英尺-磅？"
"output": "动能公式为：E=1/2mv^2，其中E表示动能，m表示物体质量，v表示物体速度。
将题目中的数据代入公式，得到：
E=1/2 x 4盎司 x (8英尺/秒)^2 = 128英尺-磅\n因此，这个物体的动能是128英尺-磅。"
"answer": "128"
***
#############
"input": "一个物体质量为4盎司，以20米/秒的速度运动，它的动能是多少焦耳？\"
"output": "动能公式为：E=1/2mv^2，其中E表示动能，m表示物体质量，v表示物体速度。
首先将质量转换为千克，1盎司=0.0283495千克，所以4盎司=0.113398千克。
然后将题目中的数据代入公式，得到：
E=1/2 x 0.113398千克 x (20米/秒)^2 = 22.6796焦耳\n因此，这个物体的动能是22.6796焦耳。"
"answer": "22.6796"
***
#############
[{'criterion': 'keywords', 'limit': ['input', 'output', 'answer']}]
['盎司']
[{'criterion': 'NULL', 'limit': 'NULL'}]
[{'criterion': 'NULL', 'limit': 'NULL'}]

1.0
1.0

====
1
模仿以下格式，出一道题目：
示例如下
包含单位：“千克”
"input": "一个物体质量为

In [26]:
# for i in range(len(score_list_2)):
#     # if np.nanmean(score_list_1[i]) < np.nanmean(score_list_2[i]) and np.nanmean(score_list_2[i]) < 1:
#     # if np.nanmean(score_list_2[i]) < 0.8:
#         matched_sample = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], labeled_data))[0]
#         res1 = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], res_list_1))[0]
#         res2 = list(filter(lambda x: x['Instruction_id'] == matched_ins_1[i], res_list_2))[0]
#         print('====')
#         print(i)
#         print(matched_sample['Instruction'])
#         print(matched_sample['Instruction_id'])
#         print('#############')
#         print(res1['model_answer'])
#         print('***')
#         for ll in matched_sample['Input_depdent_query'][0]['limit']:
#             if ll not in res1['model_answer']:
#                 print(ll)
#         print('#############')
#         print(res2['model_answer'])
#         print('***')
#         for ll in matched_sample['Input_depdent_query'][0]['limit']:
#             if ll not in res2['model_answer']:
#                 print(ll)
#         print('#############')
#         print(matched_sample['Answer_format'])
#         print(matched_sample['Input_depdent_query'][0]['limit'])
#         print(matched_sample['Task_prescribed_phrases'])
#         print(matched_sample['Count_limit'])

#         print()
#         print(score_list_1[i])
#         print(score_list_2[i])
#         print()